# Wild deserts image classification
Training is done in [Google Colab](https://colab.research.google.com/drive/1SOflnQu87P5v9klSefdetHO2b85fqTIR?usp=sharing)

### Import libraries

In [1]:
from ultralytics import YOLO
import os
import pandas as pd
import time
from glob import glob
import re
import numpy as np
from skmultilearn.model_selection import iterative_train_test_split
import shutil
import time
from PIL import Image, ExifTags



### Trained model validation/prediction
The model needs to be trained on Colab/Drone PC prior to this step

##### This block is an old training workflow but I have kept it here for reference

In [ ]:
all_detections = []
for folder in os.listdir(f"{validation_root_directory}/all_images/"):
    print(folder)
    directory = f"{validation_root_directory}/all_images/{folder}/"
    
    for counter, image in enumerate(os.listdir(directory), start=1):
        name, ext = os.path.splitext(image)
        print(f"{counter}/{len(os.listdir(directory))}")
    
        if ext == '.JPG':  
            
            predictions = model.predict(
                source=os.path.join(directory, image),
                save=True,
                save_txt=True,
                save_conf=False,
                imgsz=640,
                conf=0.1,
                iou=0.5,
                augment=True,
                project=f"{validation_root_directory}/output3/labels_images/predict",
                name = "test"
            )
            os.makedirs(f"{validation_root_directory}/output3/labels_images/predict/images", exist_ok=True)
            shutil.copy(os.path.join(directory, image), f"{validation_root_directory}/output3/labels_images/predict/images/{name}.jpg")
            if predictions[0]:
                species = model.names[int(predictions[0].boxes.cls[0])]
                conf = round(float(predictions[0].boxes.conf[0]), 3)
                path = predictions[0].path
                print(path)
                bbox = predictions[0].boxes.xywh.tolist()
                df = pd.DataFrame({'path': path, 'species': species, 'confidence': conf, 'bbox': bbox, })
                all_detections.append(df)
                newpath = f"{validation_root_directory}/output3/{species}"
            else:
                print("No detections were made.")
                newpath = f"{validation_root_directory}/output3/empty"
            
            os.makedirs(newpath, exist_ok=True)
            predictions[0].save(f"{newpath}/{name}_{conf if predictions[0] else ''}.jpg")
            
            os.makedirs(all_images_path, exist_ok=True)
            predictions[0].save(f"{all_images_path}/{name}_{conf if predictions[0] else ''}_{species if predictions[0] else 'empty'}.jpg")

timestamp = time.strftime("%Y%m%d_%H%M%S")
if all_detections:
    final_df = pd.concat(all_detections)
    final_df.to_csv(f"{validation_root_directory}/output3/detections_{timestamp}.csv", index=False)
else:
    print("No detections were made.")


##### New training block
Note that this is all better using a gpu. Probably 10x faster

In [2]:
validation_root_directory = "E:\\Wild deserts photos\\model_training\\bilbies_quolls\\exclosures"
model = YOLO("E:\\Wild deserts photos\\model_training\\runs\\detect\\day_night_no_fence_v2\\weights\\best.pt")

# model for night images is D:\\Wild deserts photos\\model_val\\day_x_five_classes_feedback_loop\\best.pt
directory = f"{validation_root_directory}"

predictions = model.predict(
    source=os.path.join(directory),
    save=False, # I don't save anything here because I want to sort into folders
    save_txt=False,
    save_conf=False,
    imgsz=640,
    conf=0.1,
    show=False, 
    stream = True,
    iou = 0.7)



Use this block if stream = TRUE (for large datasets)

In [3]:
all_detections = []
model_name = "labelled"
annotated_dir_val = "E:\\Wild deserts photos\\model_training\\bilbies_quolls\\exclosures"

for prediction in predictions:
    annotated_dir = f"{validation_root_directory}/{model_name}/labels_images/predict/annotated"
    os.makedirs(annotated_dir, exist_ok=True)
    images_dir = f"{validation_root_directory}/{model_name}/labels_images/predict/images"
    os.makedirs(images_dir, exist_ok=True)
    if prediction.boxes:  # Check if there are any boxes in the prediction
        for detection in prediction.boxes:
            species = model.names[int(detection.cls[0])]
            conf = round(float(detection.conf[0]), 3)
            path = prediction.path
            print(path)
            name = os.path.splitext(os.path.basename(prediction.path))[0]
            bbox = detection.xywh.tolist()
            labels_dir = f"{validation_root_directory}/{model_name}/labels_images/predict/labels"
            os.makedirs(labels_dir, exist_ok=True)
            labels_path = f"{labels_dir}/{name}.txt"
            prediction.save_txt(labels_path)
            path_annotated = f"{annotated_dir}/{name}_{conf}.JPG"
            prediction.save(path_annotated)
            path_original = f"{images_dir}/{name}.JPG"
            with Image.open(path) as img:
                exif = { ExifTags.TAGS.get(k, k): v for k, v in img.getexif().items() }
                Datetime = exif.get("DateTime")
                
            df = pd.DataFrame({
                'path_original': [path_original],
                'species': [species],
                'confidence': [conf],
                'bbox': [bbox],
                'path_annotated': [path_annotated],
                'label_path': [labels_path], 
                'time': [Datetime] 
            })

            all_detections.append(df)
    else:
        species = "none"
        conf = "none"
        path = prediction.path
        name = os.path.splitext(os.path.basename(prediction.path))[0]
        bbox = "none"
        labels_dir = f"{validation_root_directory}/{model_name}/labels_images/predict/labels"
        os.makedirs(labels_dir, exist_ok=True)
        labels_path = f"{labels_dir}/{name}.txt"
        with open(labels_path, 'w') as f:
            f.write('')
        path_annotated = f"{annotated_dir}/{name}_{conf}.JPG"
        prediction.save(path_annotated)
        path_original = f"{images_dir}/{name}.JPG"
        with Image.open(path) as img:
            exif = { ExifTags.TAGS.get(k, k): v for k, v in img.getexif().items() }
            Datetime = exif.get("DateTime")
        df = pd.DataFrame({
            'path_original': [path_original],
            'species': [species],
            'confidence': [conf],
            'bbox': [bbox],
            'path_annotated': [path_annotated],
            'label_path': [labels_path], 
            'time': [Datetime] 
        })

        all_detections.append(df)
        print(f"No detections for prediction {name}")
    #CHANGE THIS PATH FOR VALIDATION
    newpath = f"{annotated_dir_val}/{species}"
    os.makedirs(newpath, exist_ok=True)
    prediction.save(f"{newpath}/{name}_{conf}.jpg")

    
    shutil.copy(path, images_dir)
    
    
    


timestamp = time.strftime("%Y%m%d_%H%M%S")
if all_detections:
    final_df = pd.concat(all_detections, ignore_index=True)
    final_df.to_csv(f"{validation_root_directory}/{model_name}/detections_{timestamp}.csv", index=False)



image 1/265 E:\Wild deserts photos\model_training\bilbies_quolls\exclosures\NSCAM02_02_07_2024_22_21_19RCNX.JPG: 480x640 1 Bilby, 4295.7ms
E:\Wild deserts photos\model_training\bilbies_quolls\exclosures\NSCAM02_02_07_2024_22_21_19RCNX.JPG
image 2/265 E:\Wild deserts photos\model_training\bilbies_quolls\exclosures\NSCAM02_02_07_2024_22_21_20RCNX.JPG: 480x640 1 Bilby, 1428.7ms
E:\Wild deserts photos\model_training\bilbies_quolls\exclosures\NSCAM02_02_07_2024_22_21_20RCNX.JPG
image 3/265 E:\Wild deserts photos\model_training\bilbies_quolls\exclosures\NSCAM02_03_07_2024_21_29_48RCNX.JPG: 480x640 1 Bilby, 1371.9ms
E:\Wild deserts photos\model_training\bilbies_quolls\exclosures\NSCAM02_03_07_2024_21_29_48RCNX.JPG
image 4/265 E:\Wild deserts photos\model_training\bilbies_quolls\exclosures\NSCAM02_04_07_2024_21_47_21RCNX.JPG: 480x640 1 Bilby, 1424.5ms
E:\Wild deserts photos\model_training\bilbies_quolls\exclosures\NSCAM02_04_07_2024_21_47_21RCNX.JPG
image 5/265 E:\Wild deserts photos\model_tr

In [ ]:

predictions = model.predict(
    source="D:\\Wild deserts photos\\test/night/labels_images/predict/images/RCNX1474.JPG",
    save=False, # I don't save anything here because I want to sort into folders
    save_txt=False,
    save_conf=False,
    imgsz=640,
    conf=0.1,
    show=False, 
    stream = True,
    iou = 0.5)

In [17]:
for prediction in predictions:
    if prediction.boxes:  # Check if there are any boxes in the prediction
        for detection in prediction.boxes:
            species = model.names[int(detection.cls[0])]
            print(species)
            conf = round(float(detection.conf[0]), 3)
            print(conf)
            path = prediction.path
            print(path)
            name = os.path.splitext(os.path.basename(prediction.path))[0]
            print(name)
            bbox = detection.xywh.tolist()
            print(bbox)



image 1/1 D:\Wild deserts photos\test\night\labels_images\predict\images\RCNX1474.JPG: 480x640 2 Kangaroos, 854.6ms
Kangaroo
0.966
D:\Wild deserts photos\test\night\labels_images\predict\images\RCNX1474.JPG
RCNX1474
[[801.4869384765625, 610.0157470703125, 711.6461791992188, 404.31427001953125]]
Kangaroo
0.956
D:\Wild deserts photos\test\night\labels_images\predict\images\RCNX1474.JPG
RCNX1474
[[1256.125, 623.52880859375, 322.0238037109375, 342.7894287109375]]
Speed: 3.0ms preprocess, 854.6ms inference, 4.7ms postprocess per image at shape (1, 3, 480, 640)


In [7]:
from PIL import Image, ExifTags
with Image.open("D:\\Wild deserts photos\\test/night/labels_images/predict/images/RCNX1474.JPG") as img:
    exif = { ExifTags.TAGS.get(k, k): v for k, v in img.getexif().items() }
    time = exif.get("DateTime")
    print(time)

2024:12:18 22:19:23


In [5]:
from PIL import Image, ExifTags


In [ ]:
all_detections = []
model_name = "model_training_night_feedback_loop"  
for i in range(len(predictions)):
    #if detections
    if predictions[i].boxes:  # Check if there are any boxes in the prediction
        species = model.names[int(predictions[i].boxes.cls[0])]
        conf = round(float(predictions[i].boxes.conf[0]), 3)
        path = predictions[i].path
        # Use the correct index i instead of 1
        name = os.path.splitext(os.path.basename(predictions[i].path))[0]
        bbox = predictions[i].boxes.xywh.tolist()
        # 5) Save the label file in the "labels" folder
        labels_dir = f"{validation_root_directory}/{model_name}/labels_images/predict/labels"
        os.makedirs(labels_dir, exist_ok=True)
        labels_path = f"{labels_dir}/{name}.txt"
        predictions[i].save_txt(labels_path)
    #if no detections
    else:
        species = "none"
        conf = "none"
        path = predictions[i].path
        name = os.path.splitext(os.path.basename(predictions[i].path))[0]
        bbox = "none"
        labels_dir = f"{validation_root_directory}/{model_name}/labels_images/predict/labels"
        os.makedirs(labels_dir, exist_ok=True)
        labels_path = f"{labels_dir}/{name}.txt"
        with open(labels_path, 'w') as f:
            f.write('')
        print(f"No detections for prediction {i}")
        
    # 1) Create a folder for this species
    newpath = f"{validation_root_directory}/{model_name}/{species}"
    os.makedirs(newpath, exist_ok=True)

    # 2) Save the annotated image to the species folder
    predictions[i].save(f"{newpath}/{name}.jpg")

    # 3) Save the original image in the "images" folder
    images_dir = f"{validation_root_directory}/{model_name}/labels_images/predict/images"
    os.makedirs(images_dir, exist_ok=True)
    shutil.copy(path, images_dir)
    path_original = f"{images_dir}/{name}.JPG"

    # 4) Save the annotated image in the "annotated" folder
    annotated_dir = f"{validation_root_directory}/{model_name}/labels_images/predict/annotated"
    os.makedirs(annotated_dir, exist_ok=True)
    path_annotated = f"{annotated_dir}/{name}_{conf}.JPG"
    predictions[i].save(path_annotated)



    # 5) Build a DataFrame row (no classification check here)
    df = pd.DataFrame({
        'path_original': [path_original],
        'species': [species],
        'confidence': [conf],
        'bbox': [bbox],
        'path_annotated': [path_annotated],
        'label_path': [labels_path]
    })

    # 6) Append this to all_detections
    all_detections.append(df)
    


# Finally, save all detections to a CSV
timestamp = time.strftime("%Y%m%d_%H%M%S")
if all_detections:
    final_df = pd.concat(all_detections, ignore_index=True)
    final_df.to_csv(f"{validation_root_directory}/{model_name}/detections_{timestamp}.csv", index=False)


In [ ]:
import cv2
import matplotlib.pyplot as plt

for i in range(len(all_predictions)):
    if all_predictions[i].boxes:  # Check if there are any boxes in the prediction
        species = model.names[int(all_predictions[i].boxes.cls[0])]
        conf = round(float(all_predictions[i].boxes.conf[0]), 3)
        path = all_predictions[i].path
        name = os.path.splitext(os.path.basename(all_predictions[i].path))[0]
        bbox = all_predictions[i].boxes.xywh.tolist()
        # save the label file to the labels folder
        labels_path = f"{validation_root_directory}/output3/labels_images/predict/labels/{name}.txt"
        all_predictions[i].save_txt(labels_path)

    else:
        species = "none"
        conf = "none"
        path = all_predictions[i].path
        name = os.path.splitext(os.path.basename(all_predictions[i].path))[0]
        bbox = "none"
        labels_dir = f"{validation_root_directory}/output3/labels_images/predict/labels"
        os.makedirs(labels_dir, exist_ok=True)
        labels_path = f"{labels_dir}/{name}.txt"
        with open(labels_path, 'w') as f:
            f.write('')
        print(f"No detections for prediction {i}")
    
    # save the annotated image to the species folder
    newpath = f"{validation_root_directory}/output3/{species}"  
    os.makedirs(newpath, exist_ok=True)
    all_predictions[i].save(f"{newpath}/{name}.jpg")
        
    # save the original image to the images folder
    os.makedirs(f"{validation_root_directory}/output3/labels_images/predict/images", exist_ok=True)
    shutil.copy(path, f"{validation_root_directory}/output3/labels_images/predict/images")
    path_original = f"{validation_root_directory}/output3/labels_images/predict/images/{name}.JPG"
        
    # save the annotated image to the annotated folder
    path_annotated = f"{validation_root_directory}/output3/labels_images/predict/annotated/{name}.JPG"
    os.makedirs(f"{validation_root_directory}/output3/labels_images/predict/annotated", exist_ok=True)
    all_predictions[i].save(f"{validation_root_directory}/output3/labels_images/predict/annotated/{name}.JPG")
        
    
        
    # Display the annotated image in a pop-up window with smaller size
    img = cv2.imread(path_annotated)
    img_resized = cv2.resize(img, (800, 600))  # Resize the image to 800x600
    cv2.imshow(f"Species: {species}, Confidence: {conf}", img_resized)

    cv2.moveWindow(f"Species: {species}, Confidence: {conf}", 0, 0)  # Move the window to the top left corner
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    # Prompt user for input
    result = input("Is the detection correct? (y/n): ").strip().lower()
    
    # save the data to a dataframe
    df = pd.DataFrame({'path_original': [path_original], 
                        'species': [species], 
                        'confidence': [conf], 
                        'bbox': [bbox], 
                        "path_annotated": [path_annotated],
                        "label_path": [labels_path],
                        "results": [result]})
    
    # append the dataframe to the list of all detections
    all_detections.append(df)
    

# Save the results to a CSV file
timestamp = time.strftime("%Y%m%d_%H%M%S")
if all_detections:
    final_df = pd.concat(all_detections)
    final_df.to_csv(f"{validation_root_directory}/output3/detections_{timestamp}.csv", index=False)


### Validation using YOLO model.val()
- This is far better than the validation above

In [ ]:
model = YOLO("D:\\Wild deserts photos\\model_val\\three_class_final_21_12_2024\\best.pt")
model.val(data = "D:\\Wild deserts photos\\model_val\\three_class_final_21_12_2024\\data_v2.yaml", conf = 0.5)

In [ ]:
model = YOLO("D:\\Wild deserts photos\\model_val\\night_x_five_classes_feedback_loop\\best.pt")
model.val(data = "D:\\Wild deserts photos\\model_val\\night_x_five_classes_feedback_loop\\test_val.yaml", conf = 0.8)